In [21]:
# ruff: noqa: F401, E402
import math
import sys
import time
from bisect import bisect_left, bisect_right
from collections import Counter, deque
from functools import cmp_to_key, lru_cache, reduce
from heapq import heapify, heappop, heappush, heappushpop, nsmallest
from importlib import reload
from itertools import (
    accumulate,
    chain,
    combinations,
    islice,
    pairwise,
    permutations,
    product,
    repeat,
    starmap,
    tee,
)
from math import comb, exp, factorial, log, sqrt
from operator import add, and_, contains, indexOf, itemgetter, neg, or_, xor
from pprint import pprint
from typing import Callable, Iterable, List, Tuple

import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
from multiprocess import Process

import arrays
import combinatorics
import graphs
import lists
import mathematics
import matrix
import parsing
import search
import sequences
import sets
import stack
import strings
import trees

for m in (
    arrays,
    combinatorics,
    graphs,
    lists,
    mathematics,
    matrix,
    parsing,
    search,
    sequences,
    sets,
    stack,
    strings,
    trees,
):
    reload(m)


from binary_index_tree import BinTree
from combinatorics import fib
from parsing import format_binop_expression, parse_binop_expression
from sequences import find_if
from strings import StringView, splint

print()


In [104]:
def maximum_toys_queries(toys: List[int], queries: List[List[int]]) -> List[int]:
    n = len(toys)
    rix = [0] * n
    for i, ri in enumerate(sorted(range(n), key=lambda i: toys[i])):
        rix[ri] = i
    toys.sort()
    bit = BinTree(toys)
    ones = BinTree(1 for _ in toys)
    ans = []
    for q in queries:
        cost = q[0]
        rbroken = list(rix[i - 1] for i in islice(q, 2, None))
        for i in rbroken:
            bit.add(i, -toys[i])
            ones.add(i, -1)
        tc, l, h = -1, 0, n - 1
        while l <= h:
            m = (l + h) // 2
            if bit.sum(m) <= cost:
                tc = m
                l = m + 1
            else:
                h = m - 1
        ans.append(ones.sum(tc))
        # restore
        for i in rbroken:
            bit.add(i, toys[i])
            ones.add(i, 1)
    return ans


In [108]:
toys = [9, 7, 2, 1, 9, 4, 2, 9, 5, 8]
queries = [
    [1, 2, 6, 7],
    [9, 1, 10],
    [5, 6, 6, 7, 3, 8, 9, 10],
    [10, 5, 9, 1, 2, 3, 5],
    [3, 0],
    [4, 0],
    [2, 2, 10, 5],
    [9, 6, 4, 1, 2, 6, 3, 7],
    [10, 8, 9, 3, 6, 5, 1, 2, 8, 4],
    [10, 5, 5, 10, 4, 9, 1],
]
maximum_toys_queries(toys, queries)


[1, 4, 1, 3, 2, 2, 1, 1, 2, 3]

In [107]:
toys = [10, 8, 10, 5, 5, 7, 7, 10, 10, 6]
queries = [
    [4, 6, 1, 9, 3, 7, 2, 8],
    [8, 4, 2, 4, 1, 7],
    [6, 7, 10, 6, 4, 9, 5, 7, 1],
    [9, 5, 4, 5, 6, 2, 1],
    [1, 9, 10, 1, 9, 5, 4, 3, 8, 7, 2],
    [9, 4, 7, 10, 6, 9],
]
maximum_toys_queries(toys, queries)


[0, 1, 0, 1, 0, 1]

In [101]:
maximum_toys_queries([8, 6, 9, 2, 5], [[12, 2, 3, 4], [30, 0]])


[3, 4, 1, 0, 2]
rix: [3, 2, 4, 0, 1]
st: [2, 5, 6, 8, 9]
ones: [1, 2, 1, 4, 1]
cost: 12, brk: [4, 0]
tc: 2, ones: [0, 1, 1, 3, 0]
cost: 30, brk: []
tc: 4, ones: [1, 2, 1, 4, 1]


[2, 5]